In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
pd.options.display.width=100

In [ ]:
df = pd.read_csv('/kaggle/input/dataisbeautiful/r_dataisbeautiful_posts.csv')
df.head()

The data type of each column. 

In [ ]:
df.dtypes

The shape of the dataframe. 

In [ ]:
df.shape

Let's check for null values.  

In [ ]:
df.isna().sum()

Different columns should have a different number of unique values.

In [ ]:
df.author_flair_text.nunique()

In [ ]:
df.removed_by.nunique()

In [ ]:
df.removed_by.unique()

In [ ]:
df.total_awards_received.nunique()

In [ ]:
df.total_awards_received.unique()

In [ ]:
df.awarders.nunique()

In [ ]:
df.awarders.unique()

From the above observations, I found that

1. Most of the users didn't receive an award. So I am replacing nan with 0.
2. Removed by nan means the post is not removed so I am replacing it with "not removed"
3. author_flair_text nan will be replaced by no flair
4. Awarders, id, and full_link column is not useful for my purpose
5. Drop one row with title nan

In [ ]:
df['total_awards_received'] = df['total_awards_received'].fillna(0)
df['total_awards_received'] = df['total_awards_received'].astype(int)

In [ ]:
df['removed_by'] = df['removed_by'].fillna('not removed')
df['author_flair_text'] = df['author_flair_text'].fillna('no flair')

In [ ]:
df = df[~df.title.isna()]

In [ ]:
df.drop(columns=['id', 'awarders', 'full_link'], inplace=True)
df.head()

Title length

In [ ]:
df['title_len'] = df['title'].astype(str).apply(len)

I am changing removed column to binary. 

In [ ]:
df['removed'] = df['removed_by'].apply(lambda val: False if val=='not removed' else True  )

An overview of columns data. 

In [ ]:
df.describe()

In [ ]:
sns.distplot(df['created_utc']) 
plt.title('Post submission in Time[ Coordinated Universal Time]', 
          fontdict={'verticalalignment': 'bottom', 'fontsize':16, 'horizontalalignment': 'center'})
plt.xlabel('Time', fontsize=12)
plt.ylabel('Probability Density', fontsize=12)
plt.show()

In [ ]:
sns.distplot(df['title_len'])
plt.title('Title length', fontsize=16)
plt.xlabel('Length', fontsize=12)
plt.ylabel('Probability Density', fontsize=12)
plt.show()

In [ ]:
temp = df['over_18'].value_counts()
temp.plot(kind='bar')
plt.title('NSFW Post', fontsize=16)
plt.ylabel('Number of posts', fontsize=12)
plt.xticks(rotation=0)
for i, v in enumerate(temp):
    plt.text(i-0.1, v+3000, str(v))
plt.show()
plt.show()

In [ ]:
temp = df['removed'].value_counts()
temp.plot(kind='bar')
plt.title('Post Removed', fontsize=16)
plt.xticks(rotation=0)
plt.ylabel('Number of Posts', fontsize=12)
for i, v in enumerate(temp):
    plt.text(i-0.1, v+3000, str(v))
plt.show()

In [ ]:
df['author'].value_counts()[1:6].plot(kind='bar')
plt.title('Top 5 Author', fontsize=16)
plt.xlabel('User Name', fontsize=12)
plt.ylabel('Number of posts', fontsize=12)
plt.xticks(rotation=0)
plt.show()

In [ ]:
sns.distplot(df['num_comments'], hist=False)
plt.title('Number of comments', fontsize=16)
plt.xlabel('')
plt.show()

In [ ]:
s = df[df['removed']]['removed_by'].value_counts()
plt.stem(s.values, use_line_collection=True, markerfmt='D')
plt.xticks(range(0,5), s.index, rotation=0)
plt.title('Post removed by', fontsize=16)
plt.xlabel('Remover', fontsize=12)
plt.ylabel('Number of posts', fontsize=12)
plt.show()

Into the title

In [ ]:
stop_words = set(['NOC', 'Chart', 'Bar', 'Created', 'Confirmed', 'How', 'can', 'the'])
stopwords = STOPWORDS.update(stop_words)
plt.figure(figsize=(20, 20))
wc_1 = WordCloud(width=1600, height=800, background_color='black', stopwords=STOPWORDS, 
                 max_words=1000, min_word_length=3, min_font_size=3).generate(str(' '.join(df.title.values)))

plt.imshow(wc_1, interpolation='bilinear')
plt.axis('off')
plt.title('Most frequent words in Titles', fontsize=40, verticalalignment='bottom')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
plt.figure(figsize=(20, 20))
wc_2 = WordCloud(width=1600, height=800, background_color='black', max_words=1000, 
                 stopwords=stopwords, 
                 min_word_length=3).generate(str(' '.join(df[df['over_18']].title.values)))
plt.imshow(wc_2, interpolation='bilinear')
plt.axis('off')
plt.title('Most frequent words in NSFW Titles', fontsize=40, verticalalignment='bottom')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
plt.figure(figsize=(20, 20))
wc_3 = WordCloud(width=1600, height=800, background_color='black', 
                 stopwords=stopwords, min_word_length=3, 
                 max_words=1000).generate(str(' '.join(df[df['removed']].title.values)))
plt.imshow(wc_3, interpolation='bilinear')
plt.axis('off')
plt.title('Most Frequent words in Removed posts', fontsize=40, verticalalignment='bottom')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(20, 20))
wc_3 = WordCloud(width=600, height=400, background_color='black',
                 stopwords=stopwords, 
                 min_word_length=3).generate(str(' '.join(df['title'][df['removed'] & df['over_18']])))
plt.imshow(wc_3, interpolation='bilinear')
plt.axis('off')
plt.tight_layout()
plt.title('Most frequent words in NSFW removed posts', fontsize=40, verticalalignment='bottom')
plt.show()

In [ ]:
plt.figure(figsize=(20, 20))
wc_3 = WordCloud(width=1600, height=800, background_color='black',
                 stopwords=stopwords, min_word_length=3, 
                 max_words=2000).generate(str(' '.join(df['title'][df['removed'] & ~df['over_18']])))
plt.imshow(wc_3, interpolation='bilinear')
plt.axis('off')
plt.title('Most frequent words in SFW removed posts', fontsize=40, verticalalignment='bottom')
plt.tight_layout()
plt.show()
